## 기상청 날씨 조회 
- 참조:https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36&tabNo=2

In [1]:
#필요한 모듈 연결하기

from datetime import datetime, timedelta
import pandas as pd
import requests
import json

### 1. 날짜 입력 확인
[조건]
- 시작일/종료일 입력받기(예: 2022-01-01)
- 입력값이 10자리가 아닌경우 다시 입력 받기
- "-"를 뺀 나머지 입력 데이터가 모두 숫자인지 아닌지 확인

In [2]:
# 시작일 입력 함수
def startDate():
    # while문을 이용해 조건이 만족될때까지 계속 실행
    while True:
        # 데이터 입력
        sDate = input("조회 시작일 입력(예:2022-01-01)")
        
        # 입력값이 10자리가 아닌경우 다시 입력 받기
        if len(sDate) != 10:
            print('"2022-01-01"와 같이 "-"를 포함해 10자리로 입력하세요.')
            continue
            
        # "-"를 뺀 나머지 입력 데이터가 모두 숫자인지 아닌지 확인
        # 문자 포함시 다시 입력받기
        try:
            sdt=int(sDate.replace("-",""))
            break
        except:
            print("년, 월, 일 자리에 문자가 포함되어 있습니다.\n다시 입력하세요")
            continue
            
    return sDate
    

### 2. 종료일 입력
- 시작일과 동일한 형식으로 입력
- 종료일은 시작일 이전의 데이터 입력 불가
- 종료일이 오늘 날짜라면 -1일 진행

In [3]:
# 종료일 입력
def endDate():
    #sDate=startDate() # 시작일 입력 받기
    
    # while문을 이용해 조건이 만족될때까지 계속 실행
    while True:
        # 데이터 입력
        eDate = input("종료일 입력:(예:2022-01-01)")
        
        # 입력값이 10자리가 아닌경우 다시 입력 받기
        if len(eDate) != 10:
            print('"2022-01-01"와 같이 "-"를 포함해 10자리로 입력하세요.')
            continue
            
        # "-"를 뺀 나머지 입력 데이터가 모두 숫자인지 아닌지 확인
        # 문자 포함시 다시 입력받기
        # 입력한 종료일이 오늘 날짜인지 확인 => 오늘날짜-1 한 날짜로 세팅
        # 입력한 종료일이 시작일보다 큰거나 같은 값인지 확인 => 재입력
        try:
            edt = int(eDate.replace("-",""))
            today = datetime.today()  # 현재 시스템 날짜 가져오기
            newDate=today.strftime('%Y%m%d') # 날짜형 => 문자형으로 변경

            # 입력데이터가 현재일보다 크거나 같은 데이터인지 확인
            if edt >= int(newDate):
                # 크거나 같다면 실행 전날로 종료일 변경
                eDate=(datetime.today()-timedelta(days=1)).strftime("%Y-%m-%d")
                print('데이터는 당일 이전 자료까지만 제공합니다.')

            if int(sDate.replace("-", "")) > edt:
                print('종료일이 시작일 보다 이전 날짜입니다.\n다시 입력해주세요:')
                continue

            break
            
        except:
            print("년, 월, 일 자리에 문자가 포함되어 있습니다.\n다시 입력하세요")
            continue
            
    return sDate, eDate

### 3. 측정 지점 코드 확인
- 측정 지점의 이름을 입력하면 해당 지점 코드 출력

In [10]:
def getRegCode(inRegName="서울"):
    df_regCode = pd.read_csv("/Users/wondongchan/Desktop/multi/기상청_지역코드.csv", encoding="cp949")  # 코랩 기준
    #df_regCode = pd.read_csv("c:/pydata/기상청_지역코드.csv", encoding="cp949") # conda 기준
    regCode = df_regCode[df_regCode['지점명']==inRegName]  # 원하는 지점명 검색
    #display(regCode)
    
    if len(regCode) == 1:  # 지점명 검색 여부 확인
        # 검색 되면 코드 전달
        return int(regCode['지점'].values)
    else:
        # 검색 자료가 없으면 사용자가 강제 에러 발생
        raise Exception(f"해당 지역명은 없습니다.\n지역명 확인후 입력하세요.\n\n{inRegName}")

### 4. 날씨데이터 조회 url 만들기
- 시작일/종료일을 기준으로 지점명에 해당하는 날씨정보 조회 url 생성

In [11]:
# url 생성
def getWturl(sDate, eDate, pointID, numRows=10):
    Key = "rx1J4yPTJLMypgEsmw%2BuyunTeZzwwUbRPrwPUkkYNxyUDpxomPQu1Hjz8vXE%2BzLfrzvFQdKPXqp%2FvonN%2Bc77iA%3D%3D"
    url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=" + Key
    url = url + "&pageNo=1&dataCd=ASOS&dateCd=DAY&numOfRows=" + str(numRows)
    url = url + "&dataType=JSON" 
    url = url + "&startDt=" + str(sDate)
    url = url + "&endDt=" + str(eDate)
    url = url + "&stnIds=" + str(pointID)
    #print(url)
    
    return url

### 5. url을 이용해 실제 데이터 조회 및 DataFrame으로 저장
- 1차 조회후 전체 데이터 개수 확인
- 전체 데이터 개수를 이용한 1페이지에 조회 데이터 전체 가져오기
- 가져온 데이터에서 필요한 데이터만 DataFrame로 최종 출력하기

In [12]:
def getJsonData(url):
    soup_json=requests.get(url)
    if soup_json.status_code != 200:
        exit(f"데이터를 받지 못했습니다. 에러코드:{soup_json.status_code}" )
    json_obj=json.loads(soup_json.text)  # json 코드 읽어오기/딕셔너리 구조

    return json_obj

In [13]:
def getDataFrame(sDate, eDate, pointID):
    # 기본 url 생성
    url=getWturl(sDate.replace("-",""), eDate.replace("-",""), pointID) # 1차 url 생성
    json_obj=getJsonData(url)  # 10개의 데이터 받아오기
    numRows=json_obj["response"]["body"]["totalCount"] # 전체 데이터 조회 건수 가져오기

    # 전체 데이터를 가져오기 위한 url 생성
    url=getWturl(sDate.replace("-",""), eDate.replace("-",""), pointID, numRows) 
    json_obj=getJsonData(url)   # 1페이지에 전체 조회 데이터 가져오기

    # 데이터 정보를 가지고 있는 "item" 값만 가져오기
    items=json_obj["response"]["body"]["items"]['item'] 
    df1=pd.DataFrame(items)  # items(딕셔너리 구조)를 DataFrame으로 변경
    
    df1=df1[['stnNm', "tm", "avgTa", "minTa", "maxTa", 'sumRn']] # 원하는 열만 추출해 기존 DataFrame에 넣기
    
    return df1

### 6. 시작일을 입력하면 해당 일부터 오늘까지 데이터 전체를 조회하기
- 기준일 입력 => 해당 일을 기준으로 1월부터 12월까지 데이터 조회

In [14]:
def getDateList(sDate):    
    date_list=[]
    sdt=int(sDate[:4])  # 입력 데이터에서 4개 출력(시작년도)
    edt=datetime.today().year  # 오늘날짜에서 년도만 빼오기(종료년도)
    print(sdt, edt)  # 시작/종료 년도 확인

    date_list=[]
    cnt=0
    for i in range(sdt, edt+1):
        if cnt == 0:
            #print(sDate)
            #print(str(i)+"-12-31")
            date_list.append([sDate, str(i)+"-12-31"])
            cnt=1
        else:
            #print(str(i)+"-01-01")
            #print(str(i)+"-12-31")
            date_list.append([str(i)+"-01-01", str(i)+"-12-31"])
            
    #print(date_list)  
    return date_list

[미션] 위 코드의 결과를 이용해 날짜를 추출해 하나의 DF로 출력 

In [20]:
if __name__ == "__main__":
    sDate=startDate()
    pointName=input("지점명 입력:") # 지점명 입력
    date_list=getDateList(sDate)
    df=pd.DataFrame()
    for dt in date_list:
        pointID=getRegCode(pointName)   # 지점명에 의한 지점 코드 추출
        if int(dt[1].replace("-","")) >= int(datetime.today().strftime("%Y%m%d")):
            edt=(datetime.today()-timedelta(days=1)).strftime("%Y-%m-%d")
            print(edt)
            df1=getDataFrame(dt[0], edt, pointID)
        else:
            df1=getDataFrame(dt[0], dt[1], pointID)
        df=pd.concat([df, df1], ignore_index=True)
    df.info()    
#     df.to_csv("./날씨정보.csv", encoding = "cp949", index = False) # 코랩에 저장
    df.to_csv("/Users/wondongchan/Desktop/multi/날씨정보.csv", encoding = "cp949", index = False) # 콘다에 저장

조회 시작일 입력(예:2022-01-01)2015-01-01
지점명 입력:서울
2015 2022
2022-03-23
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2639 entries, 0 to 2638
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   stnNm   2639 non-null   object
 1   tm      2639 non-null   object
 2   avgTa   2639 non-null   object
 3   minTa   2639 non-null   object
 4   maxTa   2639 non-null   object
 5   sumRn   2639 non-null   object
dtypes: object(6)
memory usage: 123.8+ KB
